<a href="https://colab.research.google.com/github/ivanbergon/Ivan_Portfolio/blob/main/Recommendation%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de recomendação
____

Autor: Ivan Berlim Gonçalves

*atualizado em 10/08/2021*

## Objetivos:

### .Ler dados existentes e armazena-los em um dataframe
1. `songs-spotify.csv`: esse arquivo contém músicas de diferentes playlists do Spotify, com seus gêneros e respectivos artistas.
2. `user-ratings.csv`: esse arquivo contém a avaliação de cada usuário fictício para as músicas das playlists mostradas no arquivo anterior. 

### .Criar e aplicar uma função que gere notas aleatórias

### .Gerar um dataframe de acordo com as seguintes regras:
* O intervalo de notas possível para dar a uma música esteja entre 1 e 3
* Para simular o fato de que nem todos os usuários avaliam as músicas, ajustar a função `create_user_ratings` para que 23% das avaliações sejam `NaN`

### .Moldar o dataframe para a estrutura desejada (cada linha representa um usuário e cada coluna representa cada música que pode ter sido avaliada ou não)

### .Montar três cenários de recomendações:

1. Cenário 01: montar um mini sistema de recomendação com os requisitos abaixo:

  1.1. Preencher os valores faltantes com zero

  1.2. Aplicar a mesma função de ajuste de nota vista em aula

  1.3. Utilizar a distância eucliana para medir similaridade. (_Dica: use a função `euclidian_distances()` do Scikit Learn_)

  1.4. Não aplicar a variável `penalizer` durante as recomendações

  1.5. Retornando as top 5 recomendações de músicas, ordenadas da música mais similar para a menos similar.

2. Cenário 02: montar um mini sistema de recomendação com os requisitos abaixo:

  2.1. Preencher os valores faltantes com nota máxima

  2.2. Aplicar a mesma função de ajuste de nota vista em aula

  2.3. Utilizar a distância eucliana para medir similaridade. (_Dica: use a função `euclidian_distances()` do Scikit Learn_)

  2.4. Não aplica a variável `penalizer` durante as recomendações

  2.5. Retornando as top 5 recomendações de músicas, ordenadas da música mais similar para a menos similar.


3. Cenário 03: montar um mini sistema de recomendação com os requisitos abaixo:

  3.1. Preencher os valores faltantes com zero

  3.2. Usar uma função de ajuste de nota que, ao invés de subtrair a média das linhas de cada nota dada por um usuário, usa a mediana. 

  3.3. Utilizar a similaridade do cosseno

  3.4. Não aplicar a variável `penalizer` durante as recomendações

  3.5. Retornando as top 5 recomendações de músicas, ordenadas da música mais similar para a menos similar.

# .Ler dados existentes e armazena-los em um dataframe

In [ ]:
import pandas as pd           # importa biblioteca Pandas e a chama de 'pd'
import sklearn                # importa biblioteca Sklearn
import random                 # importa biblioteca Random
import numpy as np            # importa biblioteca Numpy e a chama de 'np'

## 1. `songs-spotify.csv`: esse arquivo contém músicas de diferentes playlists do Spotify, com seus gêneros e respectivos artistas.

In [ ]:
df_songs_path = "/content/drive/MyDrive/Data science/Curso Awari - Data Science/Atividade 2/songs-spotify-backup.csv"       # o caminho do arquivo de dados .csv foi relacionado à variável df_songs_path
df_songs = pd.read_csv(df_songs_path)                                                                                       # o dataset foi armazenado como df_songs através da função da biblioteca pandas pd.read_csv(caminho)
df_songs_backup = "/content/drive/MyDrive/Data science/Curso Awari - Data Science/Atividade 2/songs-spotify-backup2.csv"    # foi criado um backup chamado de df_songs_backup
df_songs.to_csv(df_songs_backup)                                                                                            # os dados foram efetivamente copiados de df_songs para df_song_backup através da função .to_csv(variável)
df_songs.info()                                                                                                             # informações básicas sobre o arquivo df_songs
print("_______________________________")
df_songs.head()                                                                                                             # 5 primeiros cabeçalhos de df_songs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      229 non-null    int64 
 1   Unnamed: 0.1    229 non-null    int64 
 2   id              229 non-null    int64 
 3   playlist_name   229 non-null    object
 4   playlist_genre  229 non-null    object
 5   artists         229 non-null    object
 6   name            229 non-null    object
dtypes: int64(3), object(4)
memory usage: 12.6+ KB
_______________________________


,Unnamed: 0,Unnamed: 0.1,id,playlist_name,playlist_genre,artists,name
0,0,0,0,Machine Gun Kelly Best Songs,punk-rock,"Machine Gun Kelly,Kellin Quinn",love race (feat. Kellin Quinn)
1,1,1,1,Machine Gun Kelly Best Songs,punk-rock,"jxdn,Machine Gun Kelly",WANNA BE (feat. Machine Gun Kelly)
2,2,2,2,Machine Gun Kelly Best Songs,punk-rock,"Machine Gun Kelly,Travis Barker",A Girl Like You
3,3,3,3,Machine Gun Kelly Best Songs,punk-rock,"Machine Gun Kelly,CORPSE",DAYWALKER! (feat. CORPSE)
4,4,4,4,Machine Gun Kelly Best Songs,punk-rock,"Trippie Redd,Travis Barker,Machine Gun Kelly,b...",PILL BREAKER (feat. blackbear & Machine Gun Ke...


## 2. `user-ratings.csv`: esse arquivo contém a avaliação de cada usuário fictício para as músicas das playlists mostradas no arquivo anterior. 

In [ ]:
df_user_ratings_path = "/content/drive/MyDrive/Data science/Curso Awari - Data Science/Atividade 2/user-ratings-backup.csv"     # o caminho do arquivo de dados .csv foi relacionado à variável df_user_ratings_path
df_user_ratings = pd.read_csv(df_user_ratings_path)                                                                             # o dataset foi armazenado como df_user_ratings através da função da biblioteca pandas pd.read_csv(caminho)
df_user_ratings_backup = "/content/drive/MyDrive/Data science/Curso Awari - Data Science/Atividade 2/user-ratings-backup2.csv"  # foi criado um backup chamado de df_user_ratings_backup
df_user_ratings.to_csv(df_user_ratings_backup)                                                                                  # os dados foram efetivamente copiados de df_songs para df_song_backup através da função .to_csv(variável)
df_user_ratings.info()                                                                                                          # informações básicas sobre o arquivo df_songs
print("-------------")
df_user_ratings.head()                                                                                                          # 5 primeiros cabeçalhos de df_songs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11450 entries, 0 to 11449
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    11450 non-null  int64  
 1   Unnamed: 0.1  11450 non-null  int64  
 2   track_id      11450 non-null  int64  
 3   name          11450 non-null  object 
 4   user_id       11450 non-null  int64  
 5   rating        9417 non-null   float64
dtypes: float64(1), int64(4), object(1)
memory usage: 536.8+ KB
-------------


,Unnamed: 0,Unnamed: 0.1,track_id,name,user_id,rating
0,0,0,0,love race (feat. Kellin Quinn),1,1.0
1,1,1,1,WANNA BE (feat. Machine Gun Kelly),1,4.0
2,2,2,2,A Girl Like You,1,NaN
3,3,3,3,DAYWALKER! (feat. CORPSE),1,3.0
4,4,4,4,PILL BREAKER (feat. blackbear & Machine Gun Ke...,1,3.0


# .Criação e aplicação de uma função que gere notas aleatórias

In [ ]:
# função para gerar códigos pseudo-aleatórios
def get_context_random_int(row, genre_lover, a, b):
  if genre_lover != None and row["playlist_genre"] == genre_lover:
    return random.randint(round((a+b)/2,0) + 1, b)

  return(random.randint(a, b))

# função para gerar notas pseudo-aleatórias com intervalo de valor definido (no caso, de 1 a 3)
def generate_ratings(df_songs, user_id, genre_lover=None, min_rating=1, max_rating=3):
  df_songs["user_id"] = user_id
  df_songs["rating"] = df_songs.apply(lambda row: get_context_random_int(row, genre_lover, min_rating, max_rating), axis=1)
  return(df_songs.drop(["playlist_name", "playlist_genre", "artists"], axis=1))

# Função para criar as notas pseudo-aleatórias, com porcentagem de valores nulos
def create_user_ratings(users_count, percentage_of_missing_values):

  dfs = []
  for i in range(users_count):
    if i > 10 and i < 30:
      genre_lover = "punk-rock"
    elif i >= 30 and i < 60:
      genre_lover = "soul-funk"
    elif i >= 60 and i < 95:
      genre_lover = "arrocha"
    else:
      genre_lover = None
  
    dfs.append(generate_ratings(df_songs.copy(), i+1, genre_lover=genre_lover))

  df_ratings = pd.concat(dfs).rename(columns={"id": "track_id"})

  # adiciona NaN aleatoriamente na coluna de rating de acordo com um percentual de corte no dataframe
  df_ratings["rating"] = df_ratings["rating"].mask(np.random.random(df_ratings.shape[0]) < percentage_of_missing_values)

  return df_ratings

# .Gerar um novo dataframe de acordo com as seguintes regras:
* O intervalo de notas possível para dar a uma música esteja entre 1 e 3
* Para simular o fato de que nem todos os usuários avaliam as músicas, ajustar a função `create_user_ratings` para que 23% das avaliações sejam `NaN`

In [ ]:
df_user_ratings2 = create_user_ratings(50,0.23)   # criação do novo data frame chamado df_user_ratings2, com 50 usuários e 23% de avaliações nulas
df_user_ratings2.head()

,Unnamed: 0,Unnamed: 0.1,track_id,name,user_id,rating
0,0,0,0,love race (feat. Kellin Quinn),1,NaN
1,1,1,1,WANNA BE (feat. Machine Gun Kelly),1,3.0
2,2,2,2,A Girl Like You,1,1.0
3,3,3,3,DAYWALKER! (feat. CORPSE),1,1.0
4,4,4,4,PILL BREAKER (feat. blackbear & Machine Gun Ke...,1,NaN


# .Moldar o dataframe para a estrutura desejada (cada linha representa um usuário e cada coluna representa cada música que pode ter sido avaliada ou não)

In [ ]:
# usada função pd.pivot_table para agregar valores dos usuários em uma linha, com os nomes das músicas nas colunas e as notas como sendo os valores
df_ratings = pd.pivot_table(df_user_ratings2, values="rating", index="user_id", columns="name")
df_ratings.head()     # a função 'pivotada' fpo chamada de df_ratings

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,1.0,1.0,2.0,2.0,2.0,3.0,1.0,1.0,2.0,NaN,NaN,2.0,2.0,NaN,NaN,1.0,NaN,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,NaN,NaN,2.0,1.0,1.0,NaN,3.0,2.0,NaN,3.0,NaN,NaN,NaN,...,1.0,1.0,1.0,2.0,NaN,3.0,3.0,3.0,3.0,2.0,NaN,NaN,2.0,2.0,3.0,3.0,1.0,3.0,NaN,NaN,3.0,3.0,3.0,2.0,1.0,NaN,3.0,1.0,3.0,2.0,NaN,1.0,NaN,3.0,NaN,1.0,NaN,3.0,2.0,1.0
2,2.0,NaN,1.0,2.0,1.0,NaN,2.0,2.0,1.0,1.0,2.0,3.0,2.0,NaN,3.0,NaN,NaN,1.0,NaN,1.0,1.0,3.0,1.0,3.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,1.0,3.0,NaN,2.0,NaN,2.0,1.0,2.0,NaN,...,1.0,1.0,1.0,NaN,2.0,1.0,2.0,1.0,2.0,3.0,3.0,1.0,2.0,1.0,NaN,3.0,1.0,1.0,2.0,NaN,2.0,1.0,1.0,1.0,NaN,1.0,3.0,2.0,2.0,2.0,NaN,3.0,3.0,3.0,NaN,1.0,3.0,3.0,2.0,2.0
3,3.0,1.0,2.0,3.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,3.0,3.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,3.0,NaN,3.0,1.0,3.0,1.0,NaN,NaN,3.0,NaN,...,3.0,3.0,NaN,2.0,1.0,3.0,3.0,2.0,1.0,3.0,1.0,2.0,NaN,1.0,2.0,1.0,NaN,2.0,2.0,2.0,NaN,3.0,1.0,2.0,2.0,NaN,3.0,2.0,3.0,3.0,2.0,1.0,NaN,1.0,2.0,1.0,2.0,1.0,2.0,3.0
4,NaN,NaN,3.0,2.0,NaN,3.0,3.0,2.0,3.0,2.0,1.0,3.0,NaN,3.0,3.0,3.0,1.0,3.0,2.0,2.0,1.0,NaN,3.0,NaN,3.0,2.0,3.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,3.0,2.0,2.0,3.0,2.0,...,NaN,1.0,NaN,1.0,3.0,1.0,3.0,NaN,NaN,NaN,1.0,2.0,NaN,2.0,3.0,NaN,1.0,1.0,1.0,NaN,2.0,NaN,1.0,3.0,1.0,2.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,1.0,1.0,NaN
5,NaN,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,NaN,2.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,NaN,3.0,3.0,3.0,NaN,NaN,2.0,3.0,2.0,1.0,NaN,NaN,1.0,3.0,2.0,NaN,...,1.0,1.0,1.0,2.0,NaN,3.0,1.0,NaN,NaN,NaN,2.0,3.0,2.0,1.0,NaN,1.0,1.0,1.0,2.0,3.0,3.0,3.0,2.0,NaN,1.0,1.0,NaN,3.0,2.0,3.0,2.0,2.0,NaN,1.0,2.0,NaN,3.0,3.0,NaN,3.0


# .Montar três cenários de recomendações:

## 1. Cenário 01: montar um mini sistema de recomendação com os requisitos abaixo:

### Preencher os valores faltantes com zero

In [ ]:
df_ratings.fillna(0.0, inplace=True) #preenche os valores NaN pelo valor zero
df_ratings.head()

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,1.0,2.0,2.0,2.0,3.0,1.0,1.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,3.0,2.0,0.0,3.0,0.0,0.0,0.0,...,1.0,1.0,1.0,2.0,0.0,3.0,3.0,3.0,3.0,2.0,0.0,0.0,2.0,2.0,3.0,3.0,1.0,3.0,0.0,0.0,3.0,3.0,3.0,2.0,1.0,0.0,3.0,1.0,3.0,2.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,3.0,2.0,1.0
2,2.0,0.0,1.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,1.0,3.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,1.0,3.0,0.0,2.0,0.0,2.0,1.0,2.0,0.0,...,1.0,1.0,1.0,0.0,2.0,1.0,2.0,1.0,2.0,3.0,3.0,1.0,2.0,1.0,0.0,3.0,1.0,1.0,2.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,3.0,2.0,2.0,2.0,0.0,3.0,3.0,3.0,0.0,1.0,3.0,3.0,2.0,2.0
3,3.0,1.0,2.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,3.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,3.0,0.0,3.0,1.0,3.0,1.0,0.0,0.0,3.0,0.0,...,3.0,3.0,0.0,2.0,1.0,3.0,3.0,2.0,1.0,3.0,1.0,2.0,0.0,1.0,2.0,1.0,0.0,2.0,2.0,2.0,0.0,3.0,1.0,2.0,2.0,0.0,3.0,2.0,3.0,3.0,2.0,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0
4,0.0,0.0,3.0,2.0,0.0,3.0,3.0,2.0,3.0,2.0,1.0,3.0,0.0,3.0,3.0,3.0,1.0,3.0,2.0,2.0,1.0,0.0,3.0,0.0,3.0,2.0,3.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,3.0,2.0,2.0,3.0,2.0,...,0.0,1.0,0.0,1.0,3.0,1.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,3.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,3.0,1.0,2.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,1.0,1.0,0.0
5,0.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,0.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,0.0,3.0,3.0,3.0,0.0,0.0,2.0,3.0,2.0,1.0,0.0,0.0,1.0,3.0,2.0,0.0,...,1.0,1.0,1.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,2.0,0.0,1.0,1.0,0.0,3.0,2.0,3.0,2.0,2.0,0.0,1.0,2.0,0.0,3.0,3.0,0.0,3.0


### Normalização

* **Garantir que a média dos dados é zero**: para isso, calculamos a média de todas as avaliações de um usuário e subtraímos a mesma de cada uma das avaliações feitas pelo usuário, ou seja, $val = x_{u, i} - \overline{x_{u}}, \; \forall \; u \in U, \; i \in I$ onde `U` é o conjunto de todos os usuários e `I` é o conjunto de todas as avaliações, $x_{u, i}$ é a avaliação que um usuário deu para um filme, $\overline{x_{u}}$ é a média das avaliações do usuário `u`. 
* **Fazer o intervalo de dados ficar entre [-1, 1]**: dividir o resultado da conta do passo 1 pelo intervalo de avaliação de cada usuário, ou seja, se a nota máxima dada por um usuário foi 5 e a mínima 1, o intervalo é igual a 4. Esse passo ajuda a garantir que todos os dados estejam na mesma escala. 

Esta função foi chamada de `adjust_rating()` e ela foi aplicada a todas as linhas do dataframe `df_ratings`, gerando o `df_ratings_std`

In [ ]:
def adjust_rating(row):
    return (row - row.mean())/(row.max()-row.min())
df_ratings1_std = df_ratings.apply(adjust_rating)
df_ratings1_std.head()       # a esta função ajustada é dada o nome de df_ratings1_std

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.253333,-0.593333,-0.266667,-0.340000,0.020000,0.253333,0.113333,0.540000,-0.220000,-0.273333,0.133333,-0.600000,-0.486667,0.166667,0.093333,-0.626667,-0.540000,-0.246667,-0.493333,0.120000,-0.260000,-0.326667,0.053333,0.093333,-0.193333,-0.140000,-0.213333,-0.540000,-0.566667,-0.033333,-0.353333,-0.253333,-0.580000,0.360000,-0.020000,-0.526667,0.326667,-0.700000,-0.633333,-0.486667,...,-0.173333,-0.233333,-0.226667,0.206667,-0.520000,0.360000,0.440000,0.393333,0.460000,0.020000,-0.660000,-0.633333,0.126667,0.013333,0.273333,0.466667,-0.280000,0.360000,-0.620000,-0.486667,0.360000,0.333333,0.380000,0.033333,-0.240000,-0.580000,0.520000,-0.260000,0.366667,0.020000,-0.620000,-0.333333,-0.653333,0.366667,-0.633333,-0.200000,-0.580000,0.473333,0.053333,-0.226667
2,0.080000,-0.593333,-0.266667,-0.006667,-0.313333,-0.413333,0.113333,0.206667,-0.220000,-0.273333,0.133333,0.400000,0.180000,-0.500000,0.426667,-0.626667,-0.540000,-0.246667,-0.493333,-0.213333,-0.260000,0.340000,-0.280000,0.426667,-0.193333,0.193333,0.120000,-0.206667,-0.233333,-0.033333,0.313333,-0.253333,0.420000,-0.640000,-0.020000,-0.526667,-0.006667,-0.366667,0.033333,-0.486667,...,-0.173333,-0.233333,-0.226667,-0.460000,0.146667,-0.306667,0.106667,-0.273333,0.126667,0.353333,0.340000,-0.300000,0.126667,-0.320000,-0.726667,0.466667,-0.280000,-0.306667,0.046667,-0.486667,0.026667,-0.333333,-0.286667,-0.300000,-0.573333,-0.246667,0.520000,0.073333,0.033333,0.020000,-0.620000,0.333333,0.346667,0.366667,-0.633333,-0.200000,0.420000,0.473333,0.053333,0.106667
3,0.413333,-0.260000,0.066667,0.326667,-0.646667,-0.413333,0.113333,-0.460000,-0.553333,-0.606667,-0.200000,-0.600000,-0.486667,-0.166667,0.426667,0.373333,-0.206667,-0.246667,0.173333,-0.213333,0.073333,0.006667,-0.280000,-0.240000,0.473333,-0.140000,-0.213333,-0.206667,0.100000,-0.366667,0.313333,-0.586667,0.420000,-0.306667,0.313333,-0.193333,-0.673333,-0.700000,0.366667,-0.486667,...,0.493333,0.433333,-0.560000,0.206667,-0.186667,0.360000,0.440000,0.060000,-0.206667,0.353333,-0.326667,0.033333,-0.540000,-0.320000,-0.060000,-0.200000,-0.613333,0.026667,0.046667,0.180000,-0.640000,0.333333,-0.286667,0.033333,0.0

### Foi utilizada a distância eucliana para medir similaridade (função `euclidian_distances()` do Scikit Learn).

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

similaridade_1 = euclidean_distances(df_ratings1_std.T)    #similaridade do cenário 1 pelo médoto da distância euclidiana
print(similaridade_1)



[[0.         3.21420458 4.10852772 ... 3.79151099 3.61601377 3.88272527]
 [3.21420458 0.         3.81488313 ... 3.88730126 3.84158532 4.50308536]
 [4.10852772 3.81488313 0.         ... 3.59907395 3.39803865 4.61013377]
 ...
 [3.79151099 3.88730126 3.59907395 ... 0.         3.61509182 4.45346307]
 [3.61601377 3.84158532 3.39803865 ... 3.61509182 0.         4.03760105]
 [3.88272527 4.50308536 4.61013377 ... 4.45346307 4.03760105 0.        ]]


In [ ]:
# similaridade do cenário 1 dentro do dataframe, comparando as diversas músicas
df_similaridade_1 = pd.DataFrame(similaridade_1, index=df_ratings1_std.columns, columns=df_ratings1_std.columns)
df_similaridade_1.head()

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27,0.000000,3.214205,4.108528,3.765634,3.322984,3.658779,4.089281,4.106634,4.352522,3.509036,3.244140,4.054079,3.894440,4.211360,3.430581,4.180377,3.972405,3.843898,3.859764,4.233202,4.358644,3.505234,3.995553,3.828548,4.074037,4.206080,4.017738,4.136826,3.783297,3.876138,3.407508,3.915780,3.511568,3.378363,3.407508,3.820704,3.853137,4.336153,4.346391,3.535534,...,3.545263,4.457453,3.522625,3.708549,3.915780,3.954182,4.132258,3.899003,3.300168,3.549335,4.144340,3.399673,4.109880,4.136558,3.684502,3.925982,3.585155,3.538989,2.990726,3.628590,3.954182,3.417601,3.865805,3.091566,3.770057,3.511568,3.603702,3.785645,3.091566,3.611402,3.689324,3.482017,3.018462,3.944335,3.858900,3.840139,4.068579,3.791511,3.616014,3.882725
5:3666,3.214205,0.000000,3.814883,4.043101,3.840139,4.033471,3.819249,4.027682,4.610134,3.770057,3.549335,3.901282,3.724991,4.001111,3.955587,3.951090,3.982183,3.740469,3.894440,3.800877,3.858612,3.800585,4.038977,3.983577,4.136558,3.551525,3.496347,4.651404,3.736903,3.899858,4.028785,3.726482,3.914645,3.590419,3.497301,4.000000,4.385836,4.148360,3.933898,3.413698,...,4.131451,4.050789,3.472111,3.711843,4.090368,4.397221,4.403282,4.545083,3.662422,3.897578,3.265986,3.354599,3.982183,4.538477,3.741657,4.363739,2.844488,3.741954,3.736903,4.067213,3.887587,3.599074,4.050789,3.253716,3.509036,3.886158,4.511959,3.887301,3.760615,3.722604,4.050789,3.505234,3.421825,3.876998,3.387559,4.074037,3.740469,3.887301,3.841585,4.503085
A Girl Like You,4.108528,3.814883,0.000000,3.980229,4.269010,3.686612,3.527983,3.434789,3.858900,4.509003,3.265986,4.447221,4.336153,3.566822,4.050789,4.159060,4.513929,3.753813,3.754701,3.982183,3.872696,3.820704,4.421161,4.105281,4.250229,3.708549,3.868965,4.538477,4.222953,3.951090,4.289781,3.998889,3.694140,4.045299,3.938697,3.473391,4.224005,4.062019,4.249183,3.639597,...,4.522536,4.403282,3.876998,3.832319,4.097696,4.311226,3.933898,4.558021,4.338202,4.216634,4.312514,4.275252,3.963444,4.431202,3.738389,4.216370,3.619699,4.206873,3.753813,3.989152,3.640513,3.741657,3.812552,3.865805,3.995553,4.482311,4.515652,3.985808,4.006938,3.829999,3.783297,4.000000,3.925982,3.979112,3.951090,4.294700,4.06639

### Retornando as top 5 recomendações de músicas, ordenadas da música mais similar para a menos similar.

In [ ]:

# faz o ajuste de similaridade e ordena de forma decrescente
def get_recommendations_decrescente(df_similaridade_1, music, rating):
    similar_score = df_similaridade_1[music]*(rating)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score
# faz o ajuste de similaridade e ordena de forma crescente
def get_recommendations_crescente(df_similaridade_1, music, rating):
    similar_score = df_similaridade_1[music]*(rating)
    similar_score = similar_score.sort_values(ascending=True)
    return similar_score

music = "Alpha Omega"
music = input("Digite o nome da música: ")
crescente = get_recommendations_crescente(df_similaridade_1, music, 3)
decrescente = get_recommendations_decrescente(df_similaridade_1, music, 3)
print("\nTOP 5 Músicas mais parecidas com a música " + music + ":\n\n", crescente[1:6])
print("\n------------------------------------------------------\n")
print("TOP 5 Músicas menos parecidas com a música " + music + ":\n\n",decrescente[0:5])

Digite o nome da música: Alpha Omega

TOP 5 Músicas mais parecidas com a música Alpha Omega:

 name
nothing inside (feat. iann dior)    9.300538
How Does It Feel?                   9.461501
Voltei Pra Vaquejada                9.601042
concert for aliens                  9.617692
Warning Shot                        9.663333
Name: Alpha Omega, dtype: float64

------------------------------------------------------

TOP 5 Músicas menos parecidas com a música Alpha Omega:

 name
Monte Carlo                       13.885964
Glass House (feat. Naomi Wild)    13.729530
Se Namorar Fosse Bom - Ao Vivo    13.605881
Casualty                          13.586022
WWIII                             13.464026
Name: Alpha Omega, dtype: float64


## Cenário 02

### Preencher os valores faltantes com nota máxima

In [ ]:
df_ratings = pd.pivot_table(df_user_ratings2, values="rating", index="user_id", columns="name")
df_ratings.fillna(3.0, inplace=True) #preenche os valores NaN pelo valor três
df_ratings.head()

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,3.0,1.0,1.0,2.0,2.0,2.0,3.0,1.0,1.0,2.0,3.0,3.0,2.0,2.0,3.0,3.0,1.0,3.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,3.0,3.0,2.0,1.0,1.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,...,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,2.0,2.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,1.0,3.0,3.0,1.0,3.0,2.0,3.0,1.0,3.0,3.0,3.0,1.0,3.0,3.0,2.0,1.0
2,2.0,3.0,1.0,2.0,1.0,3.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,1.0,3.0,1.0,1.0,3.0,1.0,3.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,1.0,3.0,3.0,2.0,3.0,2.0,1.0,2.0,3.0,...,1.0,1.0,1.0,3.0,2.0,1.0,2.0,1.0,2.0,3.0,3.0,1.0,2.0,1.0,3.0,3.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,3.0,1.0,3.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,2.0,2.0
3,3.0,1.0,2.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,1.0,3.0,3.0,1.0,3.0,3.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,3.0,3.0,3.0,1.0,3.0,1.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,2.0,1.0,3.0,3.0,2.0,1.0,3.0,1.0,2.0,3.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,3.0,1.0,2.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0
4,3.0,3.0,3.0,2.0,3.0,3.0,3.0,2.0,3.0,2.0,1.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,2.0,2.0,1.0,3.0,3.0,3.0,3.0,2.0,3.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,3.0,2.0,2.0,3.0,2.0,...,3.0,1.0,3.0,1.0,3.0,1.0,3.0,3.0,3.0,3.0,1.0,2.0,3.0,2.0,3.0,3.0,1.0,1.0,1.0,3.0,2.0,3.0,1.0,3.0,1.0,2.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,1.0,1.0,3.0
5,3.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,2.0,1.0,3.0,3.0,1.0,3.0,2.0,3.0,...,1.0,1.0,1.0,2.0,3.0,3.0,1.0,3.0,3.0,3.0,2.0,3.0,2.0,1.0,3.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,2.0,3.0,1.0,1.0,3.0,3.0,2.0,3.0,2.0,2.0,3.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0


### Normalização

* **Garantir que a média dos dados é zero**: para isso, calculamos a média de todas as avaliações de um usuário e subtraímos a mesma de cada uma das avaliações feitas pelo usuário, ou seja, $val = x_{u, i} - \overline{x_{u}}, \; \forall \; u \in U, \; i \in I$ onde `U` é o conjunto de todos os usuários e `I` é o conjunto de todas as avaliações, $x_{u, i}$ é a avaliação que um usuário deu para um filme, $\overline{x_{u}}$ é a média das avaliações do usuário `u`. 
* **Fazer o intervalo de dados ficar entre [-1, 1]**: dividir o resultado da conta do passo 1 pelo intervalo de avaliação de cada usuário, ou seja, se a nota máxima dada por um usuário foi 5 e a mínima 1, o intervalo é igual a 4. Esse passo ajuda a garantir que todos os dados estejam na mesma escala. 

Esta função foi chamada de `adjust_rating()` e ela foi aplicada a todas as linhas do dataframe `df_ratings`, gerando o `df_ratings_std`

In [ ]:
def adjust_rating(row):
    return (row - row.mean())/(row.max()-row.min())
df_ratings2_std = df_ratings.apply(adjust_rating)      # a esta função ajustada é dada o nome de df_ratings2_std
df_ratings2_std.head()     

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.253333,-0.593333,-0.266667,-0.340000,0.020000,0.253333,0.113333,0.540000,-0.220000,-0.273333,0.133333,-0.600000,-0.486667,0.166667,0.093333,-0.626667,-0.540000,-0.246667,-0.493333,0.120000,-0.260000,-0.326667,0.053333,0.093333,-0.193333,-0.140000,-0.213333,-0.540000,-0.566667,-0.033333,-0.353333,-0.253333,-0.580000,0.360000,-0.020000,-0.526667,0.326667,-0.700000,-0.633333,-0.486667,...,-0.173333,-0.233333,-0.226667,0.206667,-0.520000,0.360000,0.440000,0.393333,0.460000,0.020000,-0.660000,-0.633333,0.126667,0.013333,0.273333,0.466667,-0.280000,0.360000,-0.620000,-0.486667,0.360000,0.333333,0.380000,0.033333,-0.240000,-0.580000,0.520000,-0.260000,0.366667,0.020000,-0.620000,-0.333333,-0.653333,0.366667,-0.633333,-0.200000,-0.580000,0.473333,0.053333,-0.226667
2,0.080000,-0.593333,-0.266667,-0.006667,-0.313333,-0.413333,0.113333,0.206667,-0.220000,-0.273333,0.133333,0.400000,0.180000,-0.500000,0.426667,-0.626667,-0.540000,-0.246667,-0.493333,-0.213333,-0.260000,0.340000,-0.280000,0.426667,-0.193333,0.193333,0.120000,-0.206667,-0.233333,-0.033333,0.313333,-0.253333,0.420000,-0.640000,-0.020000,-0.526667,-0.006667,-0.366667,0.033333,-0.486667,...,-0.173333,-0.233333,-0.226667,-0.460000,0.146667,-0.306667,0.106667,-0.273333,0.126667,0.353333,0.340000,-0.300000,0.126667,-0.320000,-0.726667,0.466667,-0.280000,-0.306667,0.046667,-0.486667,0.026667,-0.333333,-0.286667,-0.300000,-0.573333,-0.246667,0.520000,0.073333,0.033333,0.020000,-0.620000,0.333333,0.346667,0.366667,-0.633333,-0.200000,0.420000,0.473333,0.053333,0.106667
3,0.413333,-0.260000,0.066667,0.326667,-0.646667,-0.413333,0.113333,-0.460000,-0.553333,-0.606667,-0.200000,-0.600000,-0.486667,-0.166667,0.426667,0.373333,-0.206667,-0.246667,0.173333,-0.213333,0.073333,0.006667,-0.280000,-0.240000,0.473333,-0.140000,-0.213333,-0.206667,0.100000,-0.366667,0.313333,-0.586667,0.420000,-0.306667,0.313333,-0.193333,-0.673333,-0.700000,0.366667,-0.486667,...,0.493333,0.433333,-0.560000,0.206667,-0.186667,0.360000,0.440000,0.060000,-0.206667,0.353333,-0.326667,0.033333,-0.540000,-0.320000,-0.060000,-0.200000,-0.613333,0.026667,0.046667,0.180000,-0.640000,0.333333,-0.286667,0.033333,0.0

### Foi utilizada a distância eucliana para medir similaridade (função `euclidian_distances()` do Scikit Learn).

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

similaridade_2 = euclidean_distances(df_ratings2_std.T)      #similaridade do cenário 2 pelo médoto da distância euclidiana
print(similaridade_2)

[[0.         3.21420458 4.10852772 ... 3.79151099 3.61601377 3.88272527]
 [3.21420458 0.         3.81488313 ... 3.88730126 3.84158532 4.50308536]
 [4.10852772 3.81488313 0.         ... 3.59907395 3.39803865 4.61013377]
 ...
 [3.79151099 3.88730126 3.59907395 ... 0.         3.61509182 4.45346307]
 [3.61601377 3.84158532 3.39803865 ... 3.61509182 0.         4.03760105]
 [3.88272527 4.50308536 4.61013377 ... 4.45346307 4.03760105 0.        ]]


In [ ]:
# similaridade do cenário 1 dentro do dataframe, comparando as diversas músicas
df_similaridade_2 = pd.DataFrame(similaridade_2, index=df_ratings2_std.columns, columns=df_ratings2_std.columns)
df_similaridade_2.head()

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27,0.000000,3.214205,4.108528,3.765634,3.322984,3.658779,4.089281,4.106634,4.352522,3.509036,3.244140,4.054079,3.894440,4.211360,3.430581,4.180377,3.972405,3.843898,3.859764,4.233202,4.358644,3.505234,3.995553,3.828548,4.074037,4.206080,4.017738,4.136826,3.783297,3.876138,3.407508,3.915780,3.511568,3.378363,3.407508,3.820704,3.853137,4.336153,4.346391,3.535534,...,3.545263,4.457453,3.522625,3.708549,3.915780,3.954182,4.132258,3.899003,3.300168,3.549335,4.144340,3.399673,4.109880,4.136558,3.684502,3.925982,3.585155,3.538989,2.990726,3.628590,3.954182,3.417601,3.865805,3.091566,3.770057,3.511568,3.603702,3.785645,3.091566,3.611402,3.689324,3.482017,3.018462,3.944335,3.858900,3.840139,4.068579,3.791511,3.616014,3.882725
5:3666,3.214205,0.000000,3.814883,4.043101,3.840139,4.033471,3.819249,4.027682,4.610134,3.770057,3.549335,3.901282,3.724991,4.001111,3.955587,3.951090,3.982183,3.740469,3.894440,3.800877,3.858612,3.800585,4.038977,3.983577,4.136558,3.551525,3.496347,4.651404,3.736903,3.899858,4.028785,3.726482,3.914645,3.590419,3.497301,4.000000,4.385836,4.148360,3.933898,3.413698,...,4.131451,4.050789,3.472111,3.711843,4.090368,4.397221,4.403282,4.545083,3.662422,3.897578,3.265986,3.354599,3.982183,4.538477,3.741657,4.363739,2.844488,3.741954,3.736903,4.067213,3.887587,3.599074,4.050789,3.253716,3.509036,3.886158,4.511959,3.887301,3.760615,3.722604,4.050789,3.505234,3.421825,3.876998,3.387559,4.074037,3.740469,3.887301,3.841585,4.503085
A Girl Like You,4.108528,3.814883,0.000000,3.980229,4.269010,3.686612,3.527983,3.434789,3.858900,4.509003,3.265986,4.447221,4.336153,3.566822,4.050789,4.159060,4.513929,3.753813,3.754701,3.982183,3.872696,3.820704,4.421161,4.105281,4.250229,3.708549,3.868965,4.538477,4.222953,3.951090,4.289781,3.998889,3.694140,4.045299,3.938697,3.473391,4.224005,4.062019,4.249183,3.639597,...,4.522536,4.403282,3.876998,3.832319,4.097696,4.311226,3.933898,4.558021,4.338202,4.216634,4.312514,4.275252,3.963444,4.431202,3.738389,4.216370,3.619699,4.206873,3.753813,3.989152,3.640513,3.741657,3.812552,3.865805,3.995553,4.482311,4.515652,3.985808,4.006938,3.829999,3.783297,4.000000,3.925982,3.979112,3.951090,4.294700,4.06639

### Retornando as top 5 recomendações de músicas, ordenadas da música mais similar para a menos similar.

In [ ]:
# faz o ajuste de similaridade e ordena de forma decrescente
def get_recommendations_decrescente(df_similaridade_2, music, rating):
    similar_score = df_similaridade_2[music]*(rating)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score
# faz o ajuste de similaridade e ordena de forma crescente
def get_recommendations_crescente(df_similaridade_2, music, rating):
    similar_score = df_similaridade_2[music]*(rating)
    similar_score = similar_score.sort_values(ascending=True)
    return similar_score

music = "Alpha Omega"
music = input("Digite o nome da música: ")
crescente = get_recommendations_crescente(df_similaridade_2, music, 3)
decrescente = get_recommendations_decrescente(df_similaridade_2, music, 3)
print("\nTOP 5 Músicas mais parecidas com a música " + music + ":\n\n", crescente[1:6])
print("\n------------------------------------------------------\n")
print("TOP 5 Músicas menos parecidas com a música " + music + ":\n\n",decrescente[0:5])

Digite o nome da música: Alpha Omega

TOP 5 Músicas mais parecidas com a música Alpha Omega:

 name
nothing inside (feat. iann dior)    9.300538
How Does It Feel?                   9.461501
Voltei Pra Vaquejada                9.601042
concert for aliens                  9.617692
Warning Shot                        9.663333
Name: Alpha Omega, dtype: float64

------------------------------------------------------

TOP 5 Músicas menos parecidas com a música Alpha Omega:

 name
Monte Carlo                       13.885964
Glass House (feat. Naomi Wild)    13.729530
Se Namorar Fosse Bom - Ao Vivo    13.605881
Casualty                          13.586022
WWIII                             13.464026
Name: Alpha Omega, dtype: float64


## Cenário 03

### Preencher os valores faltantes com zero

In [ ]:
df_ratings = pd.pivot_table(df_user_ratings2, values="rating", index="user_id", columns="name")
df_ratings.fillna(0.0, inplace=True)   #preenche os valores NaN pelo valor zero
df_ratings.head()

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,1.0,2.0,2.0,2.0,3.0,1.0,1.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,3.0,2.0,0.0,3.0,0.0,0.0,0.0,...,1.0,1.0,1.0,2.0,0.0,3.0,3.0,3.0,3.0,2.0,0.0,0.0,2.0,2.0,3.0,3.0,1.0,3.0,0.0,0.0,3.0,3.0,3.0,2.0,1.0,0.0,3.0,1.0,3.0,2.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,3.0,2.0,1.0
2,2.0,0.0,1.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,1.0,3.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,1.0,3.0,0.0,2.0,0.0,2.0,1.0,2.0,0.0,...,1.0,1.0,1.0,0.0,2.0,1.0,2.0,1.0,2.0,3.0,3.0,1.0,2.0,1.0,0.0,3.0,1.0,1.0,2.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,3.0,2.0,2.0,2.0,0.0,3.0,3.0,3.0,0.0,1.0,3.0,3.0,2.0,2.0
3,3.0,1.0,2.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,3.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,3.0,0.0,3.0,1.0,3.0,1.0,0.0,0.0,3.0,0.0,...,3.0,3.0,0.0,2.0,1.0,3.0,3.0,2.0,1.0,3.0,1.0,2.0,0.0,1.0,2.0,1.0,0.0,2.0,2.0,2.0,0.0,3.0,1.0,2.0,2.0,0.0,3.0,2.0,3.0,3.0,2.0,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0
4,0.0,0.0,3.0,2.0,0.0,3.0,3.0,2.0,3.0,2.0,1.0,3.0,0.0,3.0,3.0,3.0,1.0,3.0,2.0,2.0,1.0,0.0,3.0,0.0,3.0,2.0,3.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,3.0,2.0,2.0,3.0,2.0,...,0.0,1.0,0.0,1.0,3.0,1.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,3.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,3.0,1.0,2.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,1.0,1.0,0.0
5,0.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,0.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,0.0,3.0,3.0,3.0,0.0,0.0,2.0,3.0,2.0,1.0,0.0,0.0,1.0,3.0,2.0,0.0,...,1.0,1.0,1.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,2.0,0.0,1.0,1.0,0.0,3.0,2.0,3.0,2.0,2.0,0.0,1.0,2.0,0.0,3.0,3.0,0.0,3.0


### Normalização com a mediana

In [ ]:
import statistics
def adjust_rating(row):
    return (row - row.median())/(row.max()-row.min())  #usando a mediana
df_ratings3_std = df_ratings.apply(adjust_rating)
df_ratings3_std.head()

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.333333,-0.666667,-0.500000,-0.666667,0.000000,0.333333,0.000000,0.666667,-0.333333,-0.333333,0.000000,-0.666667,-0.666667,0.166667,0.000000,-1.000000,-0.333333,-0.333333,-0.333333,0.000000,-0.333333,-0.333333,-0.333333,0.000000,-0.333333,0.000000,-0.333333,-0.500000,-0.666667,-0.333333,-0.666667,-0.333333,-0.666667,0.333333,-0.166667,-0.500000,0.000000,-1.000000,-0.833333,-0.333333,...,0.000000,-0.333333,-0.333333,0.333333,-0.666667,0.333333,0.333333,0.333333,0.333333,-0.166667,-0.833333,-0.833333,0.000000,0.000000,0.000000,0.333333,-0.333333,0.333333,-0.666667,-0.500000,0.333333,0.166667,0.333333,0.000000,-0.333333,-0.666667,0.666667,-0.333333,0.333333,-0.166667,-0.666667,-0.666667,-0.833333,0.333333,-0.666667,-0.333333,-0.666667,0.500000,0.000000,-0.333333
2,0.000000,-0.666667,-0.500000,-0.333333,-0.333333,-0.333333,0.000000,0.333333,-0.333333,-0.333333,0.000000,0.333333,0.000000,-0.500000,0.333333,-1.000000,-0.333333,-0.333333,-0.333333,-0.333333,-0.333333,0.333333,-0.666667,0.333333,-0.333333,0.333333,0.000000,-0.166667,-0.333333,-0.333333,0.000000,-0.333333,0.333333,-0.666667,-0.166667,-0.500000,-0.333333,-0.666667,-0.166667,-0.333333,...,0.000000,-0.333333,-0.333333,-0.333333,0.000000,-0.333333,0.000000,-0.333333,0.000000,0.166667,0.166667,-0.500000,0.000000,-0.333333,-1.000000,0.333333,-0.333333,-0.333333,0.000000,-0.500000,0.000000,-0.500000,-0.333333,-0.333333,-0.666667,-0.333333,0.666667,0.000000,0.000000,-0.166667,-0.666667,0.000000,0.166667,0.333333,-0.666667,-0.333333,0.333333,0.500000,0.000000,0.000000
3,0.333333,-0.333333,-0.166667,0.000000,-0.666667,-0.333333,0.000000,-0.333333,-0.666667,-0.666667,-0.333333,-0.666667,-0.666667,-0.166667,0.333333,0.000000,0.000000,-0.333333,0.333333,-0.333333,0.000000,0.000000,-0.666667,-0.333333,0.333333,0.000000,-0.333333,-0.166667,0.000000,-0.666667,0.000000,-0.666667,0.333333,-0.333333,0.166667,-0.166667,-1.000000,-1.000000,0.166667,-0.333333,...,0.666667,0.333333,-0.666667,0.333333,-0.333333,0.333333,0.333333,0.000000,-0.333333,0.166667,-0.500000,-0.166667,-0.666667,-0.333333,-0.333333,-0.333333,-0.666667,0.000000,0.000000,0.166667,-0.666667,0.166667,-0.333333,0.000000,0

### Utilizando a similaridade do Cosseno

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similaridade_3 = cosine_similarity(df_ratings3_std.T)    # Similaridade do cosseno
print(similaridade_3)

[[ 1.          0.40765158  0.14002801 ...  0.1514384   0.20299949
   0.06865438]
 [ 0.40765158  1.          0.25152038 ...  0.11912753  0.10757898
  -0.24558651]
 [ 0.14002801  0.25152038  1.         ...  0.23326179  0.35247761
  -0.11536243]
 ...
 [ 0.1514384   0.11912753  0.23326179 ...  1.          0.19982246
  -0.25680366]
 [ 0.20299949  0.10757898  0.35247761 ...  0.19982246  1.
  -0.07247138]
 [ 0.06865438 -0.24558651 -0.11536243 ... -0.25680366 -0.07247138
   1.        ]]


In [ ]:
# similaridade do cenário 1 dentro do dataframe, comparando as diversas músicas
df_similaridade_3 = pd.DataFrame(similaridade_3, index=df_ratings3_std.columns, columns=df_ratings3_std.columns)
df_similaridade_3.head()

name,27,5:3666,A Girl Like You,A Little More,A Message From The Count,A Minha História - Ao Vivo,A Roça ou a Cidade (Da Roça Pra Cidade) - Ao Vivo,Agora Eu Pego Mesmo,All I Need (with Mahalia & Ty Dolla $ign),All Night Long,Alma de Rapariga,Alpha Omega,American Pie,Amor Falso,Amor da Despedida,Another World,At My Best (feat. Hailee Steinfeld),Atrasadinha,BBzinha - Ao Vivo,Baby Volta Pro Seu Nego,Bad Mother F*cker,Bad Things (with Camila Cabello),Basket Case,Basta Você Me Ligar - Ao Vivo,Batom De Ouro - Ao Vivo,Bebê Vem Me Procurar,Bora pro Piseiro,Breakup Season (feat. Samm Henshaw),Breathe Easy,Broke,"Bullets With Names (feat. Young Thug, RJMrLA & Lil Duke)",Burning Memories (feat. Lil Skies),Cabeça Voando,Candy (feat. Trippie Redd),Can’t Walk,Casa de Praia - Ao Vivo,Casualty,Change For Me (with Samm Henshaw),"Cheesin’ (with Cautious Clay, Remi Wolf, Still Woozy, Sophie Meiers, Claud, Melanie Faye, & HXNS)",Chupadinha - Ao Vivo,...,Tá Rocheda,U R U,Vaqueiro da Zona Rural,Voltei Pra Vaquejada,Vou Virar Fazendeiro,WANNA BE (feat. Machine Gun Kelly),WWIII,Wake + Bake,Warning Shot,Waste Love (feat. Madison Love),Wild Boy,Wild Boy - Remix,World Series,XO Tour Llif3,Young Man,Zero Saudade - Ao Vivo,acting like that (feat. Machine Gun Kelly),all I know (feat. Trippie Redd),banyan tree - interlude,bloody valentine,bloody valentine - Acoustic,body bag (feat. YUNGBLUD & Bert McCracken of The Used),can't look back,concert for aliens,drunk face,el Diablo,forget me too (feat. Halsey),hangover cure,jawbreaker,kevin and barracuda - interlude,kiss kiss,lonely,love race (feat. Kellin Quinn),my ex's best friend (with blackbear),nothing inside (feat. iann dior),play this when i'm gone,split a pill,title track,why are you here,Ô Bebê
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27,1.000000,0.407652,0.140028,0.209358,0.331308,5.751973e-02,-0.103187,-0.231957,-0.103228,0.274563,0.334497,0.051282,0.090324,-0.051782,0.302736,0.122663,0.022100,0.043492,-0.066720,-0.125436,-0.089174,0.199715,0.209855,0.015131,0.038217,-0.304444,0.013533,-0.038837,0.134377,0.162679,0.337873,0.158726,0.179969,0.297733,0.292554,0.068403,1.597735e-01,-0.067599,-0.023295,0.042133,...,0.112535,-9.825015e-02,0.190029,0.053009,0.014153,7.792865e-02,0.049417,0.149140,0.408072,0.306094,-2.562132e-17,0.368841,0.084015,-0.054133,0.157811,0.066720,0.205963,2.196940e-01,0.449823,0.239777,0.053376,0.306711,0.106018,0.416567,0.179487,0.319313,0.205879,0.167745,0.424074,0.282549,0.236139,0.325381,0.474798,0.066262,1.060184e-01,0.212486,0.026149,0.151438,0.202999,0.068654
5:3666,0.407652,1.000000,0.251520,0.104014,0.118513,-1.143090e-01,0.043942,-0.172863,-0.230788,0.168888,0.207733,0.127391,0.166680,0.057885,0.078473,0.196890,0.032939,0.100837,-0.066296,0.096942,0.151899,0.070874,0.190391,-0.060138,0.012658,0.043216,0.255500,-0.289426,0.160228,0.150869,0.115406,0.242643,-0.014902,0.215158,0.257663,-0.006797,-5.291968e-02,0.011195,0.156244,0.111640,...,-0.124245,9.762633e-02,0.217872,0.065841,-0.070318,-1.290564e-01,-0.073654,-0.148193,0.274296,0.169623,3.699223e-01,0.383679,0.143111,-0.255500,0.132684,-0.145852,0.504817,1.364369e-01,0.148989,0.056429,0.092815,0.234922,0.026336,0.360513,0.293000,0.170846,-0.159111,0.128216,0.158018,0.239811,0.086446,0.313211,0.332668,0.105345,3.160358e-01,0.117299,0.181880,0.119128,0.107579,-0.245587
A Girl Like You,0.140028,0.251520,1.000000,0.356189,-0.005567,3.624468e-02,0.315815,0.042631,0.254766,-0.060416,0.444970,-0.010771,0.092149,0.261035,0.154821,0.356739,-0.255304,0.213153,-0.078478,0.163936,0.230114,0.113860,0.298967,0.038137,0.123084,-0.091351,0.227405,-0.174024,0.073384,0.350805,0.252822,0.215421,0.220506,0.079592,0.223450,0.229881,2.639986e-01,0.288707,0.205505,-0.064897,...,-0.346678,6.706914e-02,0.171938,-0.038970,0.118913,6.406932e-18,0.249110,-0.036547,0.126047,0.207715,1.166309e-01,0.203364,0.267220,-0.102332,0.362066,0.100901,0.265334,1.439839e-17,0.211868,0.137837,0.246647,0.327479,0.205982,

### Retornando as top 5 recomendações de músicas, ordenadas da música mais similar para a menos similar.

In [ ]:
# faz o ajuste de similaridade e ordena de forma decrescente
def get_recommendations_decrescente(df_similaridade_3, music, rating):
    similar_score = df_similaridade_3[music]*(rating)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score
# faz o ajuste de similaridade e ordena de forma crescente
def get_recommendations_crescente(df_similaridade_3, music, rating):
    similar_score = df_similaridade_3[music]*(rating)
    similar_score = similar_score.sort_values(ascending=True)
    return similar_score

music = "Alpha Omega"
music = input("Digite o nome da música: ")
crescente = get_recommendations_crescente(df_similaridade_3, music, 3)
decrescente = get_recommendations_decrescente(df_similaridade_3, music, 3)
print("\nTOP 5 Músicas mais parecidas com a música " + music + ":\n\n", crescente[0:5])
print("\n------------------------------------------------------\n")
print("TOP 5 Músicas menos parecidas com a música " + music + ":\n\n",decrescente[1:6])



Digite o nome da música: Alpha Omega

TOP 5 Músicas mais parecidas com a música Alpha Omega:

 name
Se Namorar Fosse Bom - Ao Vivo   -0.662394
Só Falta Você Aceitar            -0.589855
Vaqueiro da Zona Rural           -0.570088
Agora Eu Pego Mesmo              -0.565396
Sexta-Feira - Ao Vivo            -0.531688
Name: Alpha Omega, dtype: float64

------------------------------------------------------

TOP 5 Músicas menos parecidas com a música Alpha Omega:

 name
Warning Shot                        1.296228
nothing inside (feat. iann dior)    1.272221
Safe                                1.174303
Truck Norris Interlude              1.168676
concert for aliens                  1.128763
Name: Alpha Omega, dtype: float64
